In [1]:
import numpy as np
import pandas as pd
from pyknp import Juman
from pyknp import KNP
import MeCab
import sys
from collections import Counter, defaultdict
import re
import urllib.request, urllib.error

In [3]:
tagger = MeCab.Tagger("-Owakati")
tagger.parse("")

'\n'

In [4]:
lunch_file = ["../pretest_sushitsuu_lunch.csv",
              "../pretest_sushiichi_lunch.csv",
              "../pretest_sawada_lunch.csv",
              "../pretest_mitani_lunch.csv",
              "../pretest_matsukan_lunch.csv",
              "../pretest_kiyoda_lunch.csv",
              "../pretest_imamura_lunch.csv",
              "../pretest_hatsune_lunch.csv"
              ]

In [5]:
dinner_file = ["../pretest_sushitsuu_dinner.csv",
              "../pretest_sushiichi_dinner.csv",
              "../pretest_sawada_dinner.csv",
              "../pretest_mitani_dinner.csv",
              "../pretest_matsukan_dinner.csv",
              "../pretest_kiyoda_dinner.csv",
              "../pretest_imamura_dinner.csv",
              "../pretest_hatsune_dinner.csv",
              "../pretest_aozora_dinner.csv"
              ]

In [6]:
lunch_character_file = [
    "pre_analysis_sushitsuu_lunch.csv",
    "pre_analysis_sushiichi_lunch.csv",
    "pre_analysis_sawada_lunch.csv",
    "pre_analysis_mitani_lunch.csv",
    "pre_analysis_matsukan_lunch.csv",
    "pre_analysis_kiyoda_lunch.csv",
    "pre_analysis_imamura_lunch.csv",
    "pre_analysis_hatsune_lunch.csv",
]
#特徴的なワード

In [7]:
dinner_character_file = [
    "pre_analysis_sushitsuu_dinner.csv",
    "pre_analysis_sushiichi_dinner.csv",
    "pre_analysis_sawada_dinner.csv",
    "pre_analysis_mitani_dinner.csv",
    "pre_analysis_matsukan_dinner.csv",
    "pre_analysis_kiyoda_dinner.csv",
    "pre_analysis_imamura_dinner.csv",
    "pre_analysis_aozora_dinner.csv",
]
#特徴的なワード #hatsuneは特徴的な単語がなかったので略

In [8]:
store_name = ["sushitsuu",
             "sushiichi",
             "sawada",
             "mitani",
             "matsukan",
             "kiyoda",
             "imamura",
             "hatsune",
             "aozora",
             ]

In [9]:
url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file = urllib.request.urlopen(url=url)
stop_words = [line.decode("utf-8").strip() for line in slothlib_file]
stop_words = [sw for sw in stop_words if not sw==u'']
my_stop_words = ["℃/", "℃", "kg", "訪問", "寿司", "すし", "なく", "美味しい", "美味", 
                 "～」", "○－", "○○/", ")○/", "ない", "無い","美味しく", "いい", 
                "良く", "素晴らしい", "美味しかっ", "楽しく", "旨い", "美味い", "美味しい", "良い", "おいしい",
                "素晴らしかっ", "なし", "美味し", "よい", "素晴らしく", "よし", "おいしかっ", "たのしかっ", "",
                "","","","","","","","",""]
stop_words += my_stop_words 

In [10]:
def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

In [11]:
def tokenize_ja_meishi(text, lower):
    text = replaced_text = re.sub(r"\d+", "0", text)
    node = tagger.parseToNode(str(text))
    while node:
        surface = node.surface.lower()
        if lower and node.feature.split(',')[0] in ["名詞",""] and surface not in stop_words:
            #分かち書きで取得する品詞を指定
            yield surface
            #lowerで小文字に変換 正規化
        node = node.next

In [12]:
def tokenize_meishi(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja_meishi(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [13]:
def tokenize_ja_keiyoushi(text, lower):
    text = replaced_text = re.sub(r"\d+", "0", text)
    node = tagger.parseToNode(str(text))
    while node:
        surface = node.surface.lower()
        if lower and node.feature.split(',')[0] in ["形容詞"] and surface not in stop_words:
            #分かち書きで取得する品詞を指定
            yield surface
            #lowerで小文字に変換 正規化
        node = node.next

In [14]:
def tokenize_keiyoushi(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja_keiyoushi(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [15]:
def frequent_lunch(path):
    f_words_list = [] #その店の全てのレビューの中の名詞をいれる
    f_words = [] #f_word_listのなかで多いものをいれる
    f_words_count = []
    df = pd.read_csv(path)
    for i in df["lunch_review"]:
        txt = tokenize_meishi(i, 2, 10000, True) #名詞のリスト
        for t in txt:
            f_words_list.append(t) #名詞を一つ一つ入れていく
    counter = Counter(f_words_list)
    for word, count in counter.most_common(10000):
        f_words.append(word)
        f_words_count.append(count)
    return f_words, f_words_count

In [16]:
def frequent_dinner(path):
    f_words_list = []
    f_words = []
    f_words_count = []
    df = pd.read_csv(path)
    for i in df["dinner_review"]:
        txt = tokenize_meishi(i, 2, 10000, True) #名詞のリスト
        for t in txt:
            f_words_list.append(t) #名詞を一つ一つ入れていく
    counter = Counter(f_words_list)
    for word, count in counter.most_common(10000):
        f_words.append(word)
        f_words_count.append(count)
    return f_words, f_words_count

In [17]:
many_words_lunch = [] #リストの中にリストが入っている。
many_words_lunch_c = []
for path in lunch_file:
    many_word, many_word_c = frequent_lunch(path)
    many_words_lunch.append(many_word)
    many_words_lunch_c.append(many_word_c)

In [18]:
many_words_dinner = []
many_words_dinner_c = []
for path in dinner_file:
    many_word, many_word_c = frequent_dinner(path)
    many_words_dinner.append(many_word)
    many_words_dinner_c.append(many_word)

In [18]:
many_words_lunch[0]

['握り',
 '熟成',
 'ランチ',
 'ネタ',
 'シャリ',
 '大将',
 '予約',
 '海老',
 'トロ',
 'カウンター',
 '包丁',
 '味噌汁',
 '昆布',
 '旨味',
 '仕事',
 '六本木',
 'ばら',
 'バラ',
 '種類',
 '雲丹',
 'ウニ',
 '漬け',
 'コース',
 '赤身',
 '定食',
 'お昼',
 '金目',
 '玉子',
 '写真',
 '個室',
 '江戸前',
 '締め',
 'マグロ',
 '最高',
 '出汁',
 '濃厚',
 '料理',
 '非常',
 'うに',
 '香り',
 '満足',
 '好み',
 '穴子',
 'エビ',
 '店内',
 '甘み',
 '西麻布',
 '海苔',
 '再訪',
 '焼き魚',
 'ホタテ',
 '雰囲気',
 '邪魔',
 'お願い',
 '週間',
 '貫目',
 '好き',
 '醤油',
 '通り',
 '主人',
 'ズワイガニ',
 '注文',
 '巻き',
 '店主',
 '印象',
 '最後',
 '地下',
 '全て',
 '丁寧',
 '追加',
 '提供',
 '高級',
 '一番',
 '親方',
 '感動',
 '日本',
 '値段',
 '絶品',
 'のど',
 '最初',
 'ぐろ',
 '説明',
 '東京',
 '甘味',
 'バランス',
 '研究',
 '一貫',
 'サービス',
 '素材',
 '日間',
 '旨み',
 '繊維',
 '技術',
 'カワハギ',
 'ボタン',
 '://',
 '味わい',
 '職人',
 '酒肴',
 '小鉢',
 '今日',
 '内容',
 '風味',
 'メニュー',
 '巻物',
 '事前',
 '交互',
 '軍艦',
 '有名',
 'アジ',
 '味噌',
 'えび',
 'イカ',
 '瞬間',
 'カニ',
 '気持ち',
 'イクラ',
 '代わり',
 '人気',
 '暖簾',
 '名刺',
 '女性',
 '具合',
 'ほた',
 '満席',
 'ヒラメ',
 '焼き物',
 'スタイル',
 '接客',
 '紹介',
 'ご飯',
 'シマ',
 '次回',
 'こだわり',
 'ビール',
 '一体',
 '最近',
 '

In [19]:
def lunch_kyouki(path, li):
    lunch_list = [] #同じ文中に出てくる名詞と形容詞のペアをいれる
    df = pd.read_csv(path)
    for d in df["lunch_review"]:
        for j in d.split("。"):
            txt_meishi = tokenize_meishi(j, 2, 10000, True)
            txt_keiyoushi = tokenize_keiyoushi(j, 2, 10000, True)
            for l in li:
                if l in txt_meishi:
                    for t in txt_keiyoushi:
                        lunch_list.append((l,t))
    return lunch_list

In [20]:
#名詞と形容詞の共起度の高いものを格納していく
def lunch_simpson(path, li): #liにはmany_words_lunch[i]
    lunch_list = []
    pare_simpson = defaultdict(float)
    kei_num = defaultdict(float)
    df = pd.read_csv(path)
    for d in df["lunch_review"]: #dは一本のレビュー
        for j in d.split("。"): #レビューの中の一本の文
            txt_meishi = tokenize_meishi(j, 2, 10000, True)
            txt_keiyoushi = tokenize_keiyoushi(j, 2, 10000, True)
            for t in txt_keiyoushi:
                        if kei_num[t] != None:
                            kei_num[t] += 1
                        else:
                            kei_num[t] = 1
            for l in li: #lは名詞
                if l in txt_meishi:
                    for t in txt_keiyoushi:
                        lunch_list.append((l,t))
    counter = Counter(lunch_list)
    counter_num = counter.most_common(3000)#組み合わせとその回数
    for pare in counter_num:
        if kei_num[pare[0][1]] != 0:
            #mi = min(kei_num[pare[0][1]], )
            pare_simpson[pare[0]] = float(pare[1]) / kei_num[pare[0][1]]
    return pare_simpson

In [21]:
def dinner_simpson(path, li):
    dinner_list = []
    pare_simpson = defaultdict(float)
    #名詞と形容詞のペアのシンプソン
    kei_num = defaultdict(float)
    #形容詞の出現回数
    df = pd.read_csv(path)
    for d in df["dinner_review"]:
        for j in d.split("。"):
            txt_meishi = tokenize_meishi(j, 2, 10000, True)
            txt_keiyoushi = tokenize_keiyoushi(j, 2, 10000, True)
            for l in li:
                if l in txt_meishi:
                    for t in txt_keiyoushi:
                        if kei_num[t] != None:
                            kei_num[t] += 1
                        else:
                            kei_num[t] = 1
                        dinner_list.append((l,t))
    counter = Counter(dinner_list)
    counter_num = counter.most_common(3000)
    for pare in counter_num:
        if kei_num[pare[0][1]] != 0:
            #mi = min(kei_num[pare[0][1]], )
            pare_simpson[pare[0]] = float(pare[1]) / kei_num[pare[0][1]]
    return pare_simpson

In [22]:
def lunch_pare_find(path, li):
    df = pd.read_csv(path)
    lunch_list = []
    for d in df["lunch_review"]:
        for j in d.split("。"):
            txt_meishi = tokenize_meishi(j, 2, 10000, True)
            txt_keiyoushi = tokenize_keiyoushi(j, 2, 10000, True)
            for l in li:
                if l in txt_meishi:
                    for t in txt_keiyoushi:
                        lunch_list.append((l,t))
    return lunch_list

In [23]:
def dinner_pare_find(path, li):
    df = pd.read_csv(path)
    dinner_list = []
    for d in df["dinner_review"]:
        for j in d.split("。"):
            txt_meishi = tokenize_meishi(j, 2, 10000, True)
            txt_keiyoushi = tokenize_keiyoushi(j, 2, 10000, True)
            for l in li:
                if l in txt_meishi:
                    for t in txt_keiyoushi:
                        dinner_list.append((l,t))
    return dinner_list

In [24]:
dinner_pare_all = []
for i, path in enumerate(dinner_file):
    dinner_pare_all.extend(dinner_pare_find(path, many_words_dinner[i]))
#counter = Counter(dinner_pare_all)
#counter.most_common()

In [25]:
lunch_pare_all = []
for i, path in enumerate(lunch_file):
    lunch_pare_all.extend(lunch_pare_find(path, many_words_lunch[i]))
#counter = Counter(lunch_pare_all)
#counter.most_common()

In [26]:
pare_all = lunch_pare_all + dinner_pare_all
counter = Counter(pare_all)
counter.most_common(500)

[(('シャリ', '強い'), 54),
 (('すき', 'やばし'), 44),
 (('握り', '大き'), 39),
 (('満足', '高い'), 36),
 (('シャリ', '大き'), 36),
 (('値段', '高い'), 32),
 (('握り', '高い'), 30),
 (('大将', '楽しい'), 28),
 (('旨味', '強い'), 27),
 (('ネタ', '大き'), 27),
 (('大将', '高い'), 26),
 (('ネタ', '高い'), 25),
 (('香り', '強い'), 25),
 (('香り', '高い'), 24),
 (('レベル', '高い'), 24),
 (('ネタ', '強い'), 24),
 (('包丁', '細かく'), 23),
 (('握り', '軽く'), 21),
 (('トロ', '強い'), 20),
 (('握り', '楽しい'), 19),
 (('甘み', '強い'), 19),
 (('ネタ', '大きく'), 18),
 (('ネタ', '多い'), 18),
 (('予約', 'やすい'), 18),
 (('握り', '無く'), 18),
 (('シャリ', '高い'), 18),
 (('個性', '強い'), 18),
 (('次郎', 'やばし'), 18),
 (('大将', '面白い'), 17),
 (('雲丹', '甘く'), 17),
 (('握り', '柔らかく'), 17),
 (('料理', '多い'), 17),
 (('握り', '美しい'), 17),
 (('旨み', '強い'), 17),
 (('ネタ', '楽しい'), 17),
 (('敷居', '高い'), 16),
 (('握り', '強い'), 16),
 (('トロ', '軽く'), 16),
 (('評価', '高い'), 16),
 (('握り', '良かっ'), 16),
 (('シャリ', '甘く'), 16),
 (('銀座', '高い'), 16),
 (('トロ', '大き'), 16),
 (('握り', '凄い'), 16),
 (('握り', '大きい'), 16),
 (('握り', '大きく'), 15),
 (('トロ', '凄い')

In [ ]:
sample_pare = []
for i in sample:
    sample_pare.append(i)

In [ ]:
sample_pare_n = []
for i in sample.values():
    sample_pare_n.append(i)

In [ ]:
indices = [*range(len(sample_pare_n))]
sorted_s_pare_i = sorted(indices, key=lambda i: -sample_pare_n[i])
sorted_s_pare_n = [sample_pare_n[i] for i in sorted_s_pare_i]
sorted_s_pare = [sample_pare[i] for i in sorted_s_pare_i]
print(sorted_s_pare)      # ==> [9, 7, 6, 5, 1]
print(sorted_s_pare_n)  # ==> [4, 0, 3, 1, 2]

In [ ]:
file = "pre_analysis_sushitsuu_dinner.csv"
df = pd.read_csv(file)

In [ ]:
df

In [ ]:
l = []
for i in df["0"]:
    ll = []
    for j in sorted_s_pare:
        if i == j[0]:
            ll.append(j[1])
    l.append(ll)

In [25]:
l_lunch = []
for i, path in enumerate(lunch_character_file):
    sample = lunch_simpson(lunch_file[i], many_words_lunch[i])
    #sampleには名詞と形容詞のペアとその出現回数が入っている
    sample_pare = []
    for i in sample:
        sample_pare.append(i)
        
    sample_pare_n = []
    for i in sample.values():
        sample_pare_n.append(i)
    #出現回数をsample_pare_nにいれる
    indices = [*range(len(sample_pare_n))]
    sorted_s_pare_i = sorted(indices, key=lambda i: -sample_pare_n[i])
    sorted_s_pare_n = [sample_pare_n[i] for i in sorted_s_pare_i]
    sorted_s_pare = [sample_pare[i] for i in sorted_s_pare_i]
    df = pd.read_csv(path)
    
    ll = []
    for j in df["0"]:
        lll = []
        for jj in sorted_s_pare:
            if j == jj[0]:
                lll.append(jj[1])
        df2 = pd.DataFrame({j:lll})
        ll.append(df2)  
    for i in range(len(ll)):
        if i == 0:
            df3 = pd.concat([ll[0], ll[1]], axis=1)
        elif i == 1:
            pass
        else:
            df3 = pd.concat([df3, ll[i]], axis=1)
    l_lunch.append(df3)

In [26]:
l_lunch[6]

,女将,白金,デザート,アイスクリーム,茶碗蒸し,アイス,奥様,高輪,居心地,北里,...,病院,美人,スミイカ,機会,ブレンド,ほうぼう,北里大学,大きめ,小瓶,ワサビ
0,可愛い,繁く,冷たい,おおき,厳しい,すごく,熱く,繁く,よろしい,待ち遠しく,...,遠い,暑い,おおき,NaN,弱い,おおきく,待ち遠しく,無かっ,大きかっ,NaN
1,暑い,近けれ,熱く,数少なく,柔らかい,遠い,小さ,遠く,小さ,うれしい,...,嬉しい,明るく,数少なく,NaN,程よい,っぽく,遠い,程よく,良かっ,NaN
2,高し,遠く,すごく,熱から,甘い,厳しい,優しく,難しい,よく,高い,...,高い,高く,細かく,NaN,NaN,NaN,嬉しい,温かく,NaN,NaN
3,明るく,難しい,軽く,冷たから,良し,軽く,若い,やすい,高く,NaN,...,NaN,よく,ふさわしい,NaN,NaN,NaN,高い,大き,NaN,NaN
4,おおき,熱から,多い,冷たい,高い,多い,NaN,NaN,良し,NaN,...,NaN,良し,珍しい,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,数少なく,冷たから,NaN,すごく,NaN,良し,NaN,NaN,NaN,NaN,...,NaN,NaN,遠い,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,美し,遠い,NaN,遠い,NaN,嬉しい,NaN,NaN,NaN,NaN,...,NaN,NaN,甘い,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,楽しい,優しく,NaN,柔らか,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,心地よい,すごく,NaN,嬉しい,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,遠い,やすい,NaN,高い,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in range(len(l_lunch)):
    l_lunch[i].to_csv("character_kyouki_" + store_name[i] + "_lunch.csv")

In [30]:
l_dinner = []
for i, path in enumerate(dinner_character_file):
    sample = dinner_simpson(dinner_file[i], many_words_dinner[i])
    
    sample_pare = []
    for i in sample:
        sample_pare.append(i)
        
    sample_pare_n = []
    for i in sample.values():
        sample_pare_n.append(i)
    
    indices = [*range(len(sample_pare_n))]
    sorted_s_pare_i = sorted(indices, key=lambda i: -sample_pare_n[i])
    sorted_s_pare_n = [sample_pare_n[i] for i in sorted_s_pare_i]
    sorted_s_pare = [sample_pare[i] for i in sorted_s_pare_i]
    df = pd.read_csv(path)
    
    ll = []
    for j in df["0"]:
        #print(path)
        lll = []
        for jj in sorted_s_pare:
            if j == jj[0]:
                lll.append(jj[1])
        df2 = pd.DataFrame({j:lll})
        ll.append(df2)  
    for i in range(len(ll)):
        if i == 0:
            df3 = pd.concat([ll[0], ll[1]], axis=1)
        elif i == 1:
            pass
        else:
            df3 = pd.concat([df3, ll[i]], axis=1)
    l_dinner.append(df3)

KeyError: '0'

In [ ]:
for i in range(len(l_dinner)):
    l_dinner[i].to_csv("character_kyouki_" + store_name[i] + "_dinner.csv")

In [ ]:
len(l_lunch)

In [31]:
df = pd.read_csv("pre_analysis_hatsune_dinner.csv")

In [32]:
df

,Unnamed: 0
